In [1]:
%load_ext blackcellmagic

Exercise 2.1: Parsing a FASTA file

There are packages, like Biopython and scikit-bio for processing files you encounter in bioinformatics. In this problem, though, we will work on our file I/O skills.

a) Use command line tools to investigate the FASTA file located at ~git/bootcamp/data/salmonella_spi1_region.fna. This file contains a portion of the Salmonella genome (described in Exercise 2.3).

You will notice that the first line begins with a >, signifying that the line contains information about the sequence. The remainder of the lines are the sequence itself.

b) The format of the Salmonella SPI1 region FASTA file is a common format for such files (though oftentimes FASTA files contain multiple sequences). Use the file I/O skills you have learned to write a function to read in a sequence from a FASTA file containing a single sequence (but possibly having the first line in the file beginning with >). Your function should take as input the name of the FASTA file and return two strings. First, it should return the descriptor string (which starts with >). Second, it should return a string with no gaps containing the sequence.

Test your function on the Salmonella sequence.

In [2]:
def read_sequence(fna_file):
    """Reads a .fna sequence file.
    
    Args:
        fna_file: the file location of a .fna file, given as a string
        
    Returns:
        string containing description of sequence from the first line of the file;
            this return is optional. If no description is detected, the first return will be None
        
        string containing the sequence with no gaps or newlines
    """

    with open(fna_file, "r") as f:
        f_lines = f.readlines()

        descriptor = f_lines[0].rstrip()
        # check whether descriptor line is present
        start = 0
        if descriptor[0] == ">":
            start = 1
        else:
            descriptor = None

        seq = ""
        for line in f_lines[start:]:
            seq += line.rstrip()

    return descriptor, seq

In [3]:
# I don't want to print the entire huge sequence to my notebook
salmonella_sequence_info = read_sequence("data/salmonella_spi1_region.fna")
print(salmonella_sequence_info[0], "\n\n", salmonella_sequence_info[1][:1000])

>gi|821161554|gb|CP011428.1| Salmonella enterica subsp. enterica strain YU39, complete genome, subsequence 3000000 to 3200000 

 AAAACCTTAGTAACTGGACTGCTGGGATTTTTCAGCCTGGATACGCTGGTAGATCTCTTCACGATGGACAGAAACTTCTTTCGGGGCGTTCACGCCAATACGCACCTGGTTGCCCTTCACCCCTAAAACTGTCACGGTGACCTCATCGCCAATCATGAGGGTCTCACCAACTCGACGAGTCAGAATCAGCATTCTTTGCTCCTTGAAAGATTAAAAGAGTCGGGTCTCTCTGTATCCCGGCATTATCCATCATATAACGCCAAAAAGTAAGCGATGACAAACACCTTAGGTGTAAGCAGTCATGGCATTACATTCTGTTAAACCTAAGTTTAGCCGATATACAAAACTTCAACCTGACTTTATCGTTGTCGATAGCGTTGACGTAAACGCCGCAGCACGGGCTGCGGCGCCAACGAACGCTTATAATTATTGCAATTTTGCGCTGACCCAGCCTTGTACACTGGCTAACGCTGCAGGCAGAGCTGCCGCATCCGTACCACCGGCTTGCGCCATGTCCGGACGACCGCCACCCTTACCGCCCACCTGCTGAGCGACCATCCCAATCAATTCCCCTGCTTTAACCCGGTCGGTCACATCCTTCGACACGCCCGCAATCAGAGAAACCTTACCTTCAACAACCGTTGCCAGTACGATAACGGTAGACCCCAGTTGATTTTTCAGATCATCAACCATGGTTCGCAGCATTTTCGGCTCAATACCAGCAAGCTCGCTAACCAGGAGCTTCACGCCGTTGAGATCGACCGCTTTACTGGAAAGATTTGCACTCTCCTGCGCTGCGGCCTGGTCCTTCAACTGCTGCAACTCTTTTTCCAGCTGACGTGTACGTTCCAGCACGGCACGCACTTTGTCG

Exercise 2.2: Restriction cut sites
Restriction enzymes cut DNA at specific locations called restriction sites. The sequence at a restriction site is called a recognition sequence. Here are the recognition sequences of some commonly used restriction enzymes.

HindIII: AAGCTT

EcoRI: GAATTC

KpnI: GGTACC

a) Download the FASTA file (New England Biosystems) from containing the genome of λ-phage, a bacteriophage that infect E. coli, here. [I made a text file with those contents.] Use the function you wrote in Exercise 2.1 to extract the sequence.

In [4]:
lambda_sequence = read_sequence("lambda_phage.txt")[1]
lambda_sequence[:1000]

'GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACCCTCTGAAAAGAAAGGAAACGACAGGTGCTGAAAGCGAGGCTTTTTGGCCTCTGTCGTTTCCTTTCTCTGTTTTTGTCCGTGGAATGAACAATGGAAGTCAACAAAAAGCAGCTGGCTGACATTTTCGGTGCGAGTATCCGTACCATTCAGAACTGGCAGGAACAGGGAATGCCCGTTCTGCGAGGCGGTGGCAAGGGTAATGAGGTGCTTTATGACTCTGCCGCCGTCATAAAATGGTATGCCGAAAGGGATGCTGAAATTGAGAACGAAAAGCTGCGCCGGGAGGTTGAAGAACTGCGGCAGGCCAGCGAGGCAGATCTCCAGCCAGGAACTATTGAGTACGAACGCCATCGACTTACGCGTGCGCAGGCCGACGCACAGGAACTGAAGAATGCCAGAGACTCCGCTGAAGTGGTGGAAACCGCATTCTGTACTTTCGTGCTGTCGCGGATCGCAGGTGAAATTGCCAGTATTCTCGACGGGCTCCCCCTGTCGGTGCAGCGGCGTTTTCCGGAACTGGAAAACCGACATGTTGATTTCCTGAAACGGGATATCATCAAAGCCATGAACAAAGCAGCCGCGCTGGATGAACTGATACCGGGGTTGCTGAGTGAATATATCGAACAGTCAGGTTAACAGGCTGCGGCATTTTGTCCGCGCCGGGCTTCGCTCACTGTTCAGGCCGGAGCCACAGACCGCCGTTGAATGGGCGGATGCTAATTACTATCTCCCGAAAGAATCCGCATACCAGGAAGGGCGCTGGGAAACACTGCCCTTTCAGCGGGCCATCATGAATGCGATGGGCAGCGACTACATCCGTGAGGTGAATGTGGTGAAGTCTGCCCGTGTCGGTTATTCCAAAATGCTGCTGGGTGTTTATGCCTACTTTATAGAGCATA

b) Write a function with call signature

restriction_sites(seq, recoq_seq)

that takes as arguments a sequence and the recognition sequence of a restriction enzyme sites and returns the indices of the first base of each of the restriction sites in the sequence. Use this function to find the indices of the restriction sites of λ-DNA for HindIII, EcoRI, and KpnI. Compare your results to those reported on the New England Biosystems datasheet.

In [5]:
def restriction_sites(seq, recoq_seq):
    """Finds the indices of restriction enzyme recognition sequences within a given sequence.
    
    Args:
        seq: the sequence to be searched, given as a string
        recoq_seq: the recognition sequence to be found
        
    Returns:
        a tuple of indices corresponding to the beginning of each restriction site.
            Indices are given with 0-based indexing, in contrast to 1-based indexing commonly used for sequences.
    """

    sites_list = []
    search_len = len(recoq_seq)
    for i, _ in enumerate(seq):
        if seq[i : i + search_len] == recoq_seq:
            sites_list.append(i)

    return tuple(sites_list)

In [6]:
HindIII_sites_in_lambda = restriction_sites(lambda_sequence, "AAGCTT")
EcoR1_sites_in_lambda = restriction_sites(lambda_sequence, "GAATTC")
Kpn1_sites_in_lambda = restriction_sites(lambda_sequence, "GGTACC")
print(
    "HindIII sites: ",
    HindIII_sites_in_lambda,
    "\n",
    "EcoRI sites: ",
    EcoR1_sites_in_lambda,
    "\n",
    "Kpn1 sites: ",
    Kpn1_sites_in_lambda,
)

HindIII sites:  (23129, 25156, 27478, 36894, 37458, 37583, 44140) 
 EcoRI sites:  (21225, 26103, 31746, 39167, 44971) 
 Kpn1 sites:  (17052, 18555)


I found one more HindIII site than is listed by NEB!

Exercise 2.3: Pathogenicity islands

For this and the next problem, we will work with real data from the Salmonella enterica genome. The section of the genome we will work with is in the file ~git/bootcamp/data/salmonella_spi1_region.fna. I cut it out of the full genome. It contains Salmonella pathogenicity island I (SPI1), which contains genes for surface receptors for host-pathogen interactions.

Pathogenicity islands are often marked by different GC content than the rest of the genome. We will try to locate the pathogenicity island(s) in our section of the Salmonella genome by computing GC content.

a) Use principles of TDD to write a function that divides a sequence into blocks and computes the GC content for each block, returning a tuple. The function signature should look like

gc_blocks(seq, block_size)

To be clear, if seq = 'ATGACTACGT' and block_size = 4, the blocks to be considered are

ATGA
CTAC

and the function should return (0.25, 0.5). Note that the blocks are non-overlapping and that we don't bother with the fact that end of the sequence that does not fit completely in a block.

It's easiest to do TDD in text files, but I'm going to attempt to do it here for demonstration purposes.

In [7]:
def gc_blocks(seq, block_size):
    """Computes the local GC content of a sequence using a given block size.
    
    Args:
        seq: the sequence to be evaluated, given as a string
        block_size: the length of sequence chunk to be evaluated
        
    Returns:
        a tuple of GC contents, given as decimals.
    """
    pass


def gc_content(seq):
    """Returns the GC content of a given sequence as a decimal."""
    pass

In [8]:
def test_gc_blocks_seq_len_1():
    """Tests gc_blocks for single-base sequences and block length 1."""
    assert gc_blocks("G", 1) == (1.0,)
    assert gc_blocks("C", 1) == (1.0,)
    assert gc_blocks("A", 1) == (0.0,)


def test_gc_blocks_example():
    assert gc_blocks("ATGACTACGT", 4) == (0.25, 0.5)


def test_gc_content_one_base():
    assert gc_content("G") == 1.0
    assert gc_content("C") == 1.0
    assert gc_content("A") == 0.0


def test_gc_content_empty_input():
    assert gc_content("") == 0.0


def test_gc_content_short_strings():
    assert gc_content("ATGA") == 0.25
    assert gc_content("CTAC") == 0.5


def test_gc_content_lowercase():
    assert gc_content("g") == 1.0
    assert gc_content("c") == 1.0
    assert gc_content("a") == 0.0

In [9]:
def run_all_tests_gc_blocks():
    """A function I will keep updating to run all written tests for gc_blocks."""
    test_gc_blocks_seq_len_1()
    test_gc_blocks_example()
    test_gc_content_one_base()
    test_gc_content_empty_input()
    test_gc_content_short_strings()
    test_gc_content_lowercase()

    # only reaches this point if there are no errors
    print("Success!")


run_all_tests_gc_blocks()

AssertionError: 

In [10]:
# now updating these functions


def gc_blocks(seq, block_size):
    """Computes the local GC content of a sequence using a given block size.
    
    Args:
        seq: the sequence to be evaluated, given as a string
        block_size: the length of sequence chunk to be evaluated, given as an int
        
    Returns:
        a tuple of GC contents, given as decimals.
    """

    output = []
    number_of_chunks = len(seq) // block_size

    for i in range(number_of_chunks):
        pos = i * block_size
        chunk = seq[pos : pos + block_size]
        output.append(gc_content(chunk))

    return tuple(output)


def gc_content(seq):
    """Returns the GC content of a given sequence as a decimal."""
    seq = seq.upper()
    return (seq.count("G") + seq.count("C")) / len(seq)

In [11]:
run_all_tests_gc_blocks()

ZeroDivisionError: division by zero

In [12]:
# uh oh! Let's consider that zero input.


def gc_content(seq):
    """Returns the GC content of a given sequence as a decimal.
    The GC content of an empty string is considered to be 0."""
    if len(seq) == 0:
        return 0.0

    seq = seq.upper()
    return (seq.count("G") + seq.count("C")) / len(seq)

In [13]:
run_all_tests_gc_blocks()

Success!


In [14]:
print(gc_blocks("abc", 3), "\n", gc_blocks("abc", 1))

(0.3333333333333333,) 
 (0.0, 0.0, 1.0)


b) Write a function that takes as input a sequence, block size, and a threshold GC content, and returns the original sequence where every base in a block with GC content above threshold is capitalized and every base below the threshold is lowercase. You would call the function like this:

mapped_seq = gc_map(seq, block_size, gc_thresh)

For example,

gc_map('ATGACTACGT', 4, 0.4)

returns 'atgaCTAC'. Note that bases not included in GC blocks are not included in the output sequence. Again, use principles of TDD.

In [15]:
def gc_map(seq, block_size, gc_thresh):
    """Creates a sequence map, where capital letters represent a GC content above a given sequence, 
    and other letters are lowercase.
    
    Args:
        seq: the sequence to be mapped, given as a string
        block_size: the length of sequence chunk to be evaluated, given as an int
        gc_thresh: the GC content threshhold, given as a float where 0 <= gc_thresh <= 1
    
    Returns:
        a string containing the original sequence, modified so that:
            lowercase letters represent a GC content below the given threshhold
            uppercase letters represent a GC content above the given threshhold
    """
    pass

In [16]:
def test_gc_map():
    assert gc_map("ATCG", 2, 0.5) == "atCG"

In [17]:
def run_all_tests_gc_map():
    print("Testing gc_blocks:")
    run_all_tests_gc_blocks()
    print("Testing gc_map:")
    test_gc_map()
    print("Success!")


run_all_tests_gc_map()

Testing gc_blocks:
Success!
Testing gc_map:


AssertionError: 

In [18]:
def gc_map(seq, block_size, gc_thresh):
    """Creates a sequence map, where capital letters represent a GC content above a given sequence, 
    and other letters are lowercase.
    
    Args:
        seq: the sequence to be mapped, given as a string
        block_size: the length of sequence chunk to be evaluated, given as an int
        gc_thresh: the GC content threshhold, given as a float where 0 <= gc_thresh <= 1
    
    Returns:
        a string containing the original sequence, modified so that:
            lowercase letters represent a GC content below the given threshhold
            uppercase letters represent a GC content above or equal to the given threshhold
    """
    seq_map = ""

    block_content = gc_blocks(seq, block_size)

    for i, block in enumerate(block_content):
        # I'm reusing this code, which isn't good practice,
        # but I can't figure out how else to do it since it's inside a for loop
        pos = i * block_size
        chunk = seq[pos : pos + block_size]

        if block < gc_thresh:
            seq_map += chunk.lower()
        else:
            seq_map += chunk.upper()

    return seq_map

In [19]:
run_all_tests_gc_map()

Testing gc_blocks:
Success!
Testing gc_map:
Success!


In [20]:
gc_map("ATCG", 2, 0.5)

'atCG'

In [21]:
def test_gc_map_overhanging_seq():
    assert gc_map("ATGACTACGT", 4, 0.4) == "atgaCTAC"

In [22]:
def run_all_tests_gc_map():
    print("Testing gc_blocks:")
    run_all_tests_gc_blocks()
    print("Testing gc_map:")
    test_gc_map()
    test_gc_map_overhanging_seq()
    print("Success!")


run_all_tests_gc_map()

Testing gc_blocks:
Success!
Testing gc_map:
Success!


c) Use the gc_map() function to generate a GC content map for the Salmonella sequence with block_size = 1000 and gc_thresh = 0.45. Where do you think the pathogenicity island is?

In [23]:
sal_seq = salmonella_sequence_info[1]
path_map = gc_map(sal_seq, 1000, 0.45)  # not printing this just to save space

d) Write the GC-mapped sequence (with upper and lower characters) to a new FASTA file. Use the same description line (which began with a > in the original FASTA file), and have line breaks every 60 characters in the sequence.

In [24]:
with open("salmonella_sequence_map.fasta", "w") as f:
    f.write(salmonella_sequence_info[0])
    f.write("\n")

    for i in range(len(path_map) // 60 + 1):
        f.write(path_map[i * 60 : (i + 1) * 60])
        f.write("\n")

Exercise 2.4: ORF detection

For this problem, again use principles of TDD to help you write your functions.

a) Write a function, longest_orf(), that takes a DNA sequence as input and finds the longest open reading frame (ORF) in the sequence (we will not consider reverse complements). A sequence fragment constitutes an ORF if the following are all true.

It begins with ATG.

It ends with any of TGA, TAG, or TAA.

The total number of bases is a multiple of 3.

Note that the sequence ATG may appear in the middle of an ORF. So, for example,

GGATGATGATGTAAAAC

has two ORFs, ATGATGATGTAA and ATGATGTAA. You would return the first one, since it is longer of these two.

Hint: The statement for this problem is a bit ambiguous as it is written. What other specification might you need for this function?

In [25]:
def longest_orf(seq):
    """Finds the longest open reading frame in a sequence and returns it in a string.
    
    Args:
        seq: an input DNA sequence, given as a string
    
    Returns:
        a string containing the longest open reading frame;
        that is, the longest substring of seq beginning with ATG; ending with TGA, TAG, or TAA;
        and containing a total number of bases that is a multiple of 3.
    """
    pass


def find_first_orf(seq):
    """Finds the first open reading frame in a sequence and returns its starting and ending indices in a tuple.
    
    Args:
        seq: an input DNA sequence, given as a string
    
    Returns:
        a tuple specifying the first and last indices of the first open reading frame;
        that is, the first substring of seq beginning with ATG; ending with TGA, TAG, or TAA;
        and containing a total number of bases that is a multiple of 3.
    """
    pass

In [26]:
# add to this list of inputs to run same tests on more examples
orf_test_inputs = ["ATGTAG"]
stop_codons = ("TGA", "TAG", "TAA")


def test_for_valid_longest_orfs():
    for input_seq in orf_test_inputs:
        assert longest_orf(input_seq)[:3] == "ATG"
        assert longest_orf(input_seq)[-3:] in stop_codons
        assert len(longest_orf(input_seq)) % 3 == 0


def test_for_valid_first_orfs():
    for input_seq in orf_test_inputs:
        output_indices = find_first_orf(input_seq)

        start_codon_pos = output_indices[0]
        assert input_seq[start_codon_pos : start_codon_pos + 3] == "ATG"

        stop_codon_pos = output_indices[1]
        assert input_seq[stop_codon_pos - 3 : stop_codon_pos] in stop_codons

        assert (output_indices[1] - output_indices[0]) % 3 == 0

In [27]:
def test_all_orf_finders():
    test_for_valid_first_orfs()
    test_for_valid_longest_orfs()
    print("Success!")


test_all_orf_finders()

TypeError: 'NoneType' object is not subscriptable

In [28]:
def find_first_orf(seq):
    """Finds the first open reading frame in a sequence and returns its starting and ending indices in a tuple.
    
    Args:
        seq: an input DNA sequence, given as a string
    
    Returns:
        a tuple specifying the start and end indices of the first open reading frame;
        that is, (first_index, last_index) such that ORF = seq[first_index:last_index]
        seq[last_index] is in actuality the first index after the ORF.
        
        The first ORF is defined as the first substring of seq beginning with ATG; ending with TGA, TAG, or TAA;
        and containing a total number of bases that is a multiple of 3.
        
        Returns None if no ORF is found.
    """

    seq = seq.upper()

    start_codon_pos = seq.find("ATG")
    # remember that .find returns -1 if it can't find anything!
    if start_codon_pos < 0:
        return None

    stop_codons = ("TGA", "TAG", "TAA")

    current_codon_pos = start_codon_pos + 3
    while current_codon_pos < len(seq):
        current_codon = seq[current_codon_pos : current_codon_pos + 3]
        if current_codon in stop_codons:
            # current_codon_pos is the start of the codon; include the entire codon
            return (start_codon_pos, current_codon_pos + 3)
            # note that this returns the first index of the ORF and the first index that is not the ORF
            # see docstring for details

        current_codon_pos += 3

    # if we exit that loop without breaking, then we never found a stop codon
    return None  # this happens automatically if no return is specified, but explicit is better than implicit!


def test_find_first_orf_basic():
    assert find_first_orf("ATGTGA") == (0, 6)
    assert find_first_orf("ATGTAG") == (0, 6)
    assert find_first_orf("ATGTAA") == (0, 6)
    assert find_first_orf("CATGTAG") == (1, 7)

In [29]:
def test_all_orf_finders():
    print("Testing find_first_orf: ")
    test_for_valid_first_orfs()
    test_find_first_orf_basic()
    print("Success!")

    print("Testing longest_orf: ")
    test_for_valid_longest_orfs()
    print("Success!")


test_all_orf_finders()

Testing find_first_orf: 
Success!
Testing longest_orf: 


TypeError: 'NoneType' object is not subscriptable

In [30]:
def longest_orf(seq):
    """Finds the longest open reading frame in a sequence and returns it in a string.
    Brute force algorithm that starts at the beginning and compares each successive ORF to see if it's longer.
    Returns the first longest one.
    
    Args:
        seq: an input DNA sequence, given as a string
    
    Returns:
        a string containing a single longest open reading frame;
        that is, the longest substring of seq beginning with ATG; ending with TGA, TAG, or TAA;
        and containing a total number of bases that is a multiple of 3.
    """

    current_longest_orf = ""
    i = 0

    while i <= len(seq):
        subseq = seq[i:]
        orf_pos = find_first_orf(subseq)
        # remember, i refers to seq
        # but returned indices refer to subseq; add i to refer to seq

        # what if no more ORFs?
        if orf_pos == None:
            break

        orf_len = orf_pos[1] - orf_pos[0]

        if orf_len > len(current_longest_orf):
            current_longest_orf = seq[orf_pos[0] + i : orf_pos[1] + i]

        i += orf_pos[0] + 1

    return current_longest_orf

In [31]:
def test_if_orf_is_longest():
    assert longest_orf("ATGTAGATGAAAAAATAG") == "ATGAAAAAATAG"


def test_all_orf_finders():
    print("Testing find_first_orf: ")
    test_for_valid_first_orfs()
    test_find_first_orf_basic()
    print("Success!")

    print("Testing longest_orf: ")
    test_for_valid_longest_orfs()
    test_if_orf_is_longest()
    print("Success!")


test_all_orf_finders()

Testing find_first_orf: 
Success!
Testing longest_orf: 
Success!


In [32]:
orf_test_inputs.append("LSKDJALIDJFPIVATGTAGLVKNLKDS")
test_all_orf_finders()

Testing find_first_orf: 
Success!
Testing longest_orf: 
Success!


In [33]:
# my tests can't handle empty output, though:
longest_orf("Q")

''

In [34]:
# that's acceptable output, but my tests fail on it
# maybe I should go back and throw errors with incorrect inputs!
orf_test_inputs.append("Q")
test_all_orf_finders()

Testing find_first_orf: 


TypeError: 'NoneType' object is not subscriptable

In [35]:
orf_test_inputs.remove("Q")
test_all_orf_finders()

Testing find_first_orf: 
Success!
Testing longest_orf: 
Success!


b) Use your function to find the longest ORF from the section of the Salmonella genome we are investigating.

In [36]:
longest_orf(sal_seq)

'ATGACCAACTACAGCCTGCGCGCACGCATGATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCTATAACGACCTGCAGCGTCAACTGGAAGATGCCGGCGCCAGTATTATTGAACCGCTCGCCGTCTCCAGCGAATATGGTATGAACTTACAAAACCGGGAGTCTATCGGCCAACTTATCAGCGTCCTGCACCGCAGACACTCTGATATTGTGCGGGCGATTTCCGTTTATGACGATCATAACCGTCTGTTTGTAACGTCTAATTTCCATCTGGACCCCTCACAGATGCAGCTTCCCGCCGGAGCGCCGTTTCCACGTCGTCTGAGCGTTGATCGCCACGGCGATATTATGATTCTGCGCACCCCAATTATCTCGGAGAGCTATTCGCCGGACGAGTCAGCGATTGCTGACGCGAAAAATACCAAAAATATGCTGGGGTATGTGGCGCTTGAACTGGATCTCAAGTCGGTCAGGCTACAGCAATACAAAGAGATTTTTATCTCCAGCGTGATGATGCTTTTTTGTATTGGCATTGCGCTGATCTTTGGCTGGCGGCTTATGCGCGATGTCACCGGGCCTATCCGTAATATGGTGAATACCGTTGACCGCATTCGCCGCGGACAACTGGATAGCCGGGTGGAAGGATTTATGCTGGGCGAACTGGATATGCTGAAAAACGGCATTAATTCCATGGCGATGTCGCTTGCCGCCTATCACGAAGAGATGCAGCATAATATCGATCAGGCCACTTCGGACCTGCGTGAAACCCTTGAGCAGATGGAAATCCAAAACGTTGAGCTGGATCTGGCGAAAAAGCGTGCCCAGGAAGCGGCGCGTATTAAGTCGGAGTTCCTGGCGAACATGTCGCACGAACTGCGAACGCCGCTGAACGGCGTCATTGGCTTTACCCGCCTGACATTAAAAACGGAGCTGAATCCCACCCAGCGCGACCATCTGAACACC

c) Write a function that converts a DNA sequence into a protein sequence. You can of course use the bootcamp_utils module.

In [37]:
import bootcamp_utils

In [38]:
bootcamp_utils.codons?

Type:        dict
String form: {'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',  <...> , 'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}
Length:      64
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [39]:
def translate_DNA(seq):
    """Given a DNA sequence as a string, returns a single-letter-code amino acid sequence."""

    i = 0
    protein = ""

    while i + 3 <= len(seq):
        protein += bootcamp_utils.codons[seq[i : i + 3]]
        i += 3

    return protein

In [40]:
translate_DNA("CATATTTGAACTCATGAACGCGAA")

'HI*THERE'

d) Translate the longest ORF you generated in part (b) into a protein sequence and perform a BLAST search. Search for the protein sequence (a blastp query). What gene is it?

In [41]:
translate_DNA(longest_orf(sal_seq))

'MTNYSLRARMMILILAPTVLIGLLLSIFFVVHRYNDLQRQLEDAGASIIEPLAVSSEYGMNLQNRESIGQLISVLHRRHSDIVRAISVYDDHNRLFVTSNFHLDPSQMQLPAGAPFPRRLSVDRHGDIMILRTPIISESYSPDESAIADAKNTKNMLGYVALELDLKSVRLQQYKEIFISSVMMLFCIGIALIFGWRLMRDVTGPIRNMVNTVDRIRRGQLDSRVEGFMLGELDMLKNGINSMAMSLAAYHEEMQHNIDQATSDLRETLEQMEIQNVELDLAKKRAQEAARIKSEFLANMSHELRTPLNGVIGFTRLTLKTELNPTQRDHLNTIERSANNLLAIINDVLDFSKLEAGKLILESIPFPLRNTLDEVVTLLAHSSHDKGLELTLNIKNDVPDNVIGDPLRLQQVITNLVGNAIKFTESGNIDILVEKRALSNTKVQIEVQIRDTGIGIPERDQSRLFQAFRQADASISRRHGGTGLGLVITQKLVNEMGGDISFHSQPNRGSTFWFHINLDLNPNVIIDGPSTACLAGKRLAYVEPNATAAQCTLDLLSDTPVEVVYSPTFSALPLAHYDIMILSVPVTFREPLTMQHERLAKAASMTDFLLLALPCHAQINAEKLKQGGAAACLLKPLTSTRLLPALTEYCQLNHHPEPLLMDTSKITMTVMAVDDNPANLKLIGALLEDKVQHVELCDSGHQAVDRAKQMQFDLILMDIQMPDMDGIRACELIHQLPHQQQTPVIAVTAHAMAGQKEKLLSAGMNDYLAKPIEEEKLHNLLLRYKPGANVAARLMAPEPAEFIFNPNATLDWQLALRQAAGKPDLARDMLQMLIDFLPEVRNKIEEQLVGENPNGLVDLVHKLHGSCGYSGVPRMKNLCQLIEQQLRSGVHEEELEPEFLELLDEMDNVAREAKKILG*'

That's two-component sensor histidine kinase BarA [Salmonella]

e) [Bonus challenge] Modify your function to return the n longest ORFs. Compute the five longest ORFs for the Salmonella genome section we are working with. Perform BLAST searches on them. What are they?

In [42]:
def longest_orf(seq, n=1):
    """Finds the n longest open reading frame(s) in a sequence and returns them in a list of strings.
    Brute force algorithm that starts at the beginning and compares each successive ORF to see if it's longer.
    For ORFs of equivalent length, returns the first one.
    
    Args:
        seq: an input DNA sequence, given as a string
        n: number of longest ORFs to return
    
    Returns:
        a string or list of strings containing the n longest open reading frame(s);
        that is, the longest substring of seq beginning with ATG; ending with TGA, TAG, or TAA;
        and containing a total number of bases that is a multiple of 3.
    """

    longest_orf_list = [""] * n
    i = 0

    while i <= len(seq):
        subseq = seq[i:]
        orf_pos = find_first_orf(subseq)
        # remember, i refers to seq
        # but returned indices refer to subseq; add i to refer to seq

        # what if no more ORFs?
        if orf_pos == None:
            break

        orf_len = orf_pos[1] - orf_pos[0]

        for index, listed_orf in enumerate(longest_orf_list):
            if orf_len > len(listed_orf):
                longest_orf_list[index] = seq[orf_pos[0] + i : orf_pos[1] + i]
                break

        i += orf_pos[0] + 1

    return longest_orf_list


# gotta rewrite my tests to accept list output


def test_for_valid_longest_orfs():
    for input_seq in orf_test_inputs:
        for orf in longest_orf(input_seq):
            assert orf[:3] == "ATG"
            assert orf[-3:] in stop_codons
            assert len(orf) % 3 == 0


def test_if_orf_is_longest():
    assert longest_orf("ATGTAGATGAAAAAATAG") == ["ATGAAAAAATAG"]

In [43]:
test_all_orf_finders()

Testing find_first_orf: 
Success!
Testing longest_orf: 
Success!


In [44]:
top_five = longest_orf(sal_seq, n=5)

In [45]:
for item in top_five:
    print(item[:100])

ATGACCAACTACAGCCTGCGCGCACGCATGATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCT
ATGATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCTATAACGACCTGCAGCGTCAACTGGAAG
ATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCTATAACGACCTGCAGCGTCAACTGGAAGATG
ATGAACTTACAAAACCGGGAGTCTATCGGCCAACTTATCAGCGTCCTGCACCGCAGACACTCTGATATTGTGCGGGCGATTTCCGTTTATGACGATCATA
ATGCAGCTTCCCGCCGGAGCGCCGTTTCCACGTCGTCTGAGCGTTGATCGCCACGGCGATATTATGATTCTGCGCACCCCAATTATCTCGGAGAGCTATT


In [46]:
# unfortunately, all of these ORFs are contained in the first one, rather than being separate genes.
top_five[4] in top_five[0]

True

In [47]:
def check_if_enclosed(tup_1, tup_2):
    """Given two two-tuples tuples specifying indices, checks whether one is entirely enclosed by the other.
    Returns True if enclosed."""

    len_tup_1 = tup_1[1] - tup_1[0]
    len_tup_2 = tup_2[1] - tup_2[0]

    if len_tup_2 < len_tup_1:
        tup_1, tup_2 = tup_2, tup_1

    if tup_1[0] > tup_2[0] and tup_1[1] <= tup_2[1]:
        return True
    else:
        return False

In [48]:
check_if_enclosed((2, 4), (1, 5))

True

In [49]:
def test_two_outputs_longest_orf():
    assert sorted(longest_orf("ATGTAGATGAAAAAATAG", n=2)) == ["ATGAAAAAATAG", "ATGTAG"]


def test_exclude_enclosed():
    assert sorted(longest_orf("ATGATGTAG", n=2)) == ["", "ATGATGTAG"]


def test_all_orf_finders():
    print("Testing find_first_orf: ")
    test_for_valid_first_orfs()
    test_find_first_orf_basic()
    print("Success!")

    print("Testing longest_orf: ")
    test_for_valid_longest_orfs()
    test_if_orf_is_longest()
    test_two_outputs_longest_orf()
    test_exclude_enclosed()
    print("Success!")


test_all_orf_finders()

Testing find_first_orf: 
Success!
Testing longest_orf: 


AssertionError: 

In [50]:
def len_from_indices(indices):
    """Given a two-tuple of indices, returns the length of the indicated sequence."""
    return indices[1] - indices[0]


def longest_orf(seq, n=1):
    """Finds the n longest open reading frame(s) in a sequence and returns them in a list of strings.
    Brute force algorithm that starts at the beginning and compares each successive ORF to see if it's longer.
    For ORFs of equivalent length, returns the first one.
    
    Args:
        seq: an input DNA sequence, given as a string
        n: number of longest ORFs to return
    
    Returns:
        a string or list of strings containing the n longest open reading frame(s);
        that is, the longest substring of seq beginning with ATG; ending with TGA, TAG, or TAA;
        and containing a total number of bases that is a multiple of 3.
    """

    longest_orf_indices = [(0, 0),] * n
    i = 0

    # create list of incides
    while i <= len(seq):
        subseq = seq[i:]
        orf_pos = find_first_orf(subseq)
        # remember, i refers to seq
        # but returned indices refer to subseq; add i to refer to seq

        # what if no more ORFs?
        if orf_pos == None:
            break

        orf_pos_plus_i = (orf_pos[0] + i, orf_pos[1] + i)

        orf_len = len_from_indices(orf_pos)

        enclosed = False
        for item, orf_indices in enumerate(longest_orf_indices):
            if check_if_enclosed(orf_pos_plus_i, orf_indices):
                enclosed = True
                break

        if enclosed == False:
            for item, orf_indices in enumerate(longest_orf_indices):
                listed_orf_len = len_from_indices(orf_indices)
                if orf_len > listed_orf_len:
                    longest_orf_indices[item] = orf_pos_plus_i
                    longest_orf_indices.sort(key=len_from_indices)
                    break

        i += orf_pos[0] + 1

    longest_orf_list = []
    # translate indices to ORFs
    for orf in longest_orf_indices:
        longest_orf_list.append(seq[orf[0] : orf[1]])

    return longest_orf_list

In [51]:
test_all_orf_finders()

Testing find_first_orf: 
Success!
Testing longest_orf: 
Success!


In [52]:
top_five = longest_orf(sal_seq, n=5)

In [53]:
for item in top_five:
    print(item[:100])

ATGCCACATTTTAATCCTGTTCCTGTATCGAATAAAAAATTCGTCTTTGATGATTTCATACTCAACATGGACGGCTCCCTGCTACGCTCAGAAAAGAAAG
ATGAAAAAAATCAGCTTACCGAAAATCGGTATCCGCCCGGTGATTGATGGACGTCGTATGGGCGTACGCGAGTCGCTCGAAGAGCAAACCATGAACATGG
ATGTCGTATACACCCATGAGCGATCTTGGACAGCAAGGCTTGTTCGATATCACTCGTACCTTATTGCAGCAGCCCGATTTGGCGTCGCTCAGTGAAGCGC
ATGAATGAGTCATTTGATAAGGACTTCTCCAACCACACCCCGATGATGCAGCAGTATCTCAAGCTGAAAGCCCAGCATCCGGAGATCCTGCTCTTTTATC
ATGACCAACTACAGCCTGCGCGCACGCATGATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCT


In [54]:
for i in range(5):
    for j in range(5):
        print(f"top_five[{i}] in top_five[{j}]: ", top_five[i] in top_five[j])

top_five[0] in top_five[0]:  True
top_five[0] in top_five[1]:  False
top_five[0] in top_five[2]:  False
top_five[0] in top_five[3]:  False
top_five[0] in top_five[4]:  False
top_five[1] in top_five[0]:  False
top_five[1] in top_five[1]:  True
top_five[1] in top_five[2]:  False
top_five[1] in top_five[3]:  False
top_five[1] in top_five[4]:  False
top_five[2] in top_five[0]:  False
top_five[2] in top_five[1]:  False
top_five[2] in top_five[2]:  True
top_five[2] in top_five[3]:  False
top_five[2] in top_five[4]:  False
top_five[3] in top_five[0]:  False
top_five[3] in top_five[1]:  False
top_five[3] in top_five[2]:  False
top_five[3] in top_five[3]:  True
top_five[3] in top_five[4]:  False
top_five[4] in top_five[0]:  False
top_five[4] in top_five[1]:  False
top_five[4] in top_five[2]:  False
top_five[4] in top_five[3]:  False
top_five[4] in top_five[4]:  True


In [55]:
proteins = []

for seq in top_five:
    proteins.append(translate_DNA(seq))

In [56]:
proteins

['MPHFNPVPVSNKKFVFDDFILNMDGSLLRSEKKVNIPPKEYAVLVILLEAAGEIVSKNTLLDQVWGDAEVNEESLTRCIYALRRILSEDKEHRYIETLYGQGYRFNRPVVVVSPPAPQPTTHTLAILPFQMQDQVQSESLHYSIVKGLSQYAPFGLSVLPVTITKNCRSVKDILELMDQLRPDYYISGQMIPDGNDNIVQIEIVRVKGYHLLHQESIKLIEHQPASLLQNKIANLLLRCIPGLRWDTKQISELNSIDSTMVYLRGKHELNQYTPYSLQQALKLLTQCVNMSPNSIAPYCALAECYLSMAQMGIFDKQNAMIKAKEHAIKATELDHNNPQALGLLGLINTIHSEYIVGSLLFKQANLLSPISADIKYYYGWNLFMAGQLEEALQTINECLKLDPTRAAAGITKLWITYYHTGIDDAIRLGDELRSQHLQDNPILLSMQVMFLSLKGKHELARKLTKEISTQEITGLIAVNLLYAEYCQNSERALPTIREFLESEQRIDNNPGLLPLVLVAHGEAIAEKMWNKFKNEDNIWFKRWKQDPRLIKLR*',
 'MKKISLPKIGIRPVIDGRRMGVRESLEEQTMNMAKATAALITEKIRHACGAQVECVIADTCIAGMAESAACEEKFSSQNVGVTITVTPCWCYGSETIDMDPMRPKAIWGFNGTERPGAVYLAAALAAHSQKGIPAFSIYGHDVQDADDTSIPADVEEKLLRFARAGLAVASMKGKSYLSVGGVSMGIAGSIVDHNFFESWLGMKVQAVDMTELRRRIDQKIYDEAELEMALAWADKNFRYGEDQNASQYKRNEAQNRAVLKESLLMAMCIRDMMQGNKTLADKGLVEESLGYNAIAAGFQGQRHWTDQYPNGDTAEALLNSSFDWNGVREPFVVATENDSLNGVAMLFGHQLTGTAQIFADVRTYWSPEAVERVTGQALSGLAEHGIIHLINSGSAALDGACKQRDSEGKPTMKPHWEISQQEADACLAATEWCPAIHE